<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>








# Mining your GeoDataFrame

This final session covers two main topics:

* The computation of spatial distances

* The computation of spatial indicators from the spatial data

Let me first check what I have in the map geopackage file from Brazil, already reprojected:

In [11]:
!pip install fiona
from fiona import listlayers

In [12]:
brazilMapsLink='https://github.com/CienciaDeDatosEspacial/GeoDataFrame_Analytics/raw/main/maps/brazilMaps_5641.gpkg'

#layers in maps
listlayers(brazilMapsLink)

['country',
 'cities',
 'rivers',
 'states',
 'municipalities',
 'airports',
 'border']

Let's read in the data:

In [ ]:
import geopandas as gpd
# guardando cada capa
states=gpd.read_file(brazilMapsLink,layer='states')
municipalities=gpd.read_file(brazilMapsLink,layer='municipalities')
airports=gpd.read_file(brazilMapsLink,layer='airports')
rivers=gpd.read_file(brazilMapsLink,layer='rivers')
border=gpd.read_file(brazilMapsLink,layer='border')

Now, we are going to add more data. In this [link](https://msi.nga.mil/Publications/WPI) we find the  World Port Index (Pub 150), which contains several data on major ports and terminals world-wide. Let me read the **UpdatedPub150.csv** from a GitHub repo:

In [ ]:
import pandas as pd

portsFileLink="https://github.com/CienciaDeDatosEspacial/GeoDataFrame_Analytics/raw/main/data/UpdatedPub150.csv"
infoseaports=pd.read_csv(portsFileLink)

#columns available (so many)
infoseaports.columns.to_list()

Let's do some preprocessing:

In [ ]:
#rename
infoseaports.rename(columns={'Main Port Name':'portName'},inplace=True)
#keep few columns
infoseaports=infoseaports.loc[:,['portName', 'Country Code','Latitude', 'Longitude']]

# we have now
infoseaports.info()

In [ ]:
# some rows
infoseaports.head()

Let's turn those points into projected spatial object (GDF of points):

In [ ]:
#spatial points (unprojected)
seaports=gpd.GeoDataFrame(data=infoseaports.copy(),
                           geometry=gpd.points_from_xy(infoseaports.Longitude,
                                                       infoseaports.Latitude),
                          crs=4326)# notice it is unprojected

# keep Brazil
seaports_bra=seaports[seaports['Country Code']=='Brazil'].copy()

# reset indexes
seaports_bra.reset_index(drop=True, inplace=True)

# reprojecting
seaports_bra_5641=seaports_bra.to_crs(5641) # projected crs

Let me plot seaports along with the _large_ airports:

In [ ]:
# subsetting
largeAirports=airports[airports['kind']=='large_airport'] #can't use "airports.type"
largeAirports.reset_index(drop=True, inplace=True)

#plotting
base=largeAirports.plot(color='red',marker="^")
seaports_bra_5641.plot(ax=base,alpha=0.5,markersize=3)

# I. MINING SPATIAL LOCATION

## Distance between points

The easiest way to understand distance is to compute how far two coordinates are from each other.

You have the seaports:

In [ ]:
seaports_bra_5641.head()

... and the large airports:

In [ ]:
largeAirports.head()

If both GDFs have **the same projected CRS**, we can use the *distance()* function. In this case, just two selected points:

In [ ]:
# distance between 'Guarulhos' and 'Dtse / Gegua Oil Terminal'
largeAirports.iloc[0].geometry.distance(seaports_bra_5641.iloc[0].geometry)/1000  # in km

What about computing all possible distances between those GDFs?

In [ ]:
#try 1: default
seaports_bra_5641.geometry.apply\
(lambda g: largeAirports.geometry.distance(g)/1000)

In [ ]:
# try 2: see names (change indexes)

seaports_bra_5641.set_index('portName').geometry.apply\
(lambda g: largeAirports.set_index('name').geometry.distance(g)/1000)

In [ ]:
#try 3: reorder previous output

seaports_bra_5641.set_index('portName').geometry.apply\
(lambda g: largeAirports.set_index('name').geometry.distance(g)/1000).\
sort_index(axis=0).sort_index(axis=1)

Let's keep the last one:

In [ ]:
distanceMatrixKM_sea_air= seaports_bra_5641.set_index('portName').geometry.apply\
                          (lambda g: largeAirports.set_index('name').geometry.distance(g)/1000).\
                          sort_index(axis=0).sort_index(axis=1)

This a data frame (pandas), and the names of the facilities are row and column indexes. From this distance matrix we can make some key queries:

In [ ]:
# the mean distance from a seaport to all the large airports (sorted)
distanceMatrixKM_sea_air.mean(axis=1).sort_values(ascending=True) #axis=0?

Let's compute more stats:

In [ ]:
SomeStats=pd.DataFrame()
SomeStats['mean']=distanceMatrixKM_sea_air.mean(axis=1)
SomeStats['min']=distanceMatrixKM_sea_air.min(axis=1)
SomeStats['max']=distanceMatrixKM_sea_air.max(axis=1)

# see some
SomeStats.head(10)

We can also use **idxmax** to get the particular locations:

In [ ]:
# farthest airport to each seaport
distanceMatrixKM_sea_air.idxmax(axis=1)

In [ ]:
# farthest seaport to each airport
distanceMatrixKM_sea_air.idxmax(axis=0)

In [ ]:
# closest airport to each seaport
distanceMatrixKM_sea_air.idxmin(axis=1)

In [ ]:
# closest seaport to each airport
distanceMatrixKM_sea_air.idxmin(axis=0)

### Exercises Part I

<div class="alert-warning">
    
Exercises 1,2,3,4 for this part must read you data from GitHub links, and the coding and results must be published using GitHub Pages

</div>

### Exercise 1

<div class="alert-success">
    
1. Use two maps of points from your country.

2. Compute the distance matrix for both maps.

3. Select one row of the distance matrix, and plot the two points with the minimal distance on top of the country of your choosing.
</div>

## Distance between line and point

Let's take a look at the rivers we have:

In [ ]:
rivers.head()

In [ ]:
#keep one:

rivers[rivers.NAME.str.contains('Grande')]

You can see that distance works between these two elements:

In [ ]:
# distance from each airport to Rio Grande
rivers[rivers.NAME.str.contains('Grande')].iloc[0].geometry.distance(largeAirports.set_index('name').geometry)/1000

Based on what we did previously, let's compute all the distances:

In [ ]:
distanceMatrixKM_riv_air=rivers.set_index('NAME').geometry.apply\
(lambda g: largeAirports.set_index('name').geometry.distance(g)/1000).\
sort_index(axis=0).sort_index(axis=1)
distanceMatrixKM_riv_air

Here, we see one row (river), that tells the distance to each column (large airport):

In [ ]:
distanceMatrixKM_riv_air.loc['Rio Grande, South America'].sort_values()

Let's try a simple plot of the river and the airports:

In [ ]:
base=largeAirports.explore(color='red',marker_kwds=dict(radius=10))
rivers[rivers.NAME.str.contains('Grande')].explore(m=base)

Now, let's focus on the rivers that belong to a system:

In [ ]:
rivers[~rivers.SYSTEM.isna()]

Let's dissolve the ones that belong to a system into a multiline:

In [ ]:
systems=rivers.dissolve(by='SYSTEM')
systems

Let's do some basic formatting:

In [ ]:
# format the GDF:

systems.reset_index(drop=False,inplace=True)
systems.drop(columns='NAME',inplace=True)

# we have
systems

Another distance matrix:

In [ ]:
distanceMatrixKM_sys_air=systems.set_index('SYSTEM').geometry.apply\
(lambda g: largeAirports.set_index('name').geometry.distance(g)/1000).\
sort_index(axis=0).sort_index(axis=1)

distanceMatrixKM_sys_air

This time, let me get all the minimum distances:

In [ ]:
mins=distanceMatrixKM_sys_air.idxmin(axis="columns") # same as axis=1
mins

In [ ]:
# one of them
mins.iloc[1]

Let's see now:

In [ ]:
base=systems.explore()
# the closest
largeAirports[largeAirports.name.isin(mins)].explore(m=base,color='red',marker_kwds=dict(radius=10))
# NOT the closest
largeAirports[~largeAirports.name.isin(mins)].explore(m=base,color='blue',marker_kwds=dict(radius=5))

### Exercise 2

<div class="alert-success">
    
1. Use a map of points and a map of lines from your country.

2. Compute the distance matrix for both.

3. Select one line of the distance matrix, and plot the closests and the farthest point to that line.
    
    
</div>

## Polygon to point

Let me create some **convex hull**s for our Brazilian system of rivers:

In [ ]:
# polygon for each system
systems.convex_hull

In [ ]:
# see them
systems.convex_hull.plot()

Now, a GDF for the hulls:

In [ ]:
systems_hulls=systems.convex_hull.to_frame()
systems_hulls['system']=['Amazon', 'Parana']
systems_hulls.rename(columns={0:'geometry'},inplace=True)
systems_hulls=systems_hulls.set_geometry('geometry')
systems_hulls.crs="EPSG:5641"
systems_hulls

Next, the distance matrix:

In [ ]:
distanceMatrixKM_sysHull_air=systems_hulls.set_index('system').geometry.apply\
(lambda g: largeAirports.set_index('name').geometry.distance(g)/1000).\
sort_index(axis=0).sort_index(axis=1)

distanceMatrixKM_sysHull_air

All the minimal differences:

In [ ]:
mins=distanceMatrixKM_sysHull_air.idxmin(axis="columns")
mins

In [ ]:
# plotting
base=systems_hulls.explore()
largeAirports[largeAirports.name.isin(mins)].explore(m=base,color='red',marker_kwds=dict(radius=10))
largeAirports[~largeAirports.name.isin(mins)].explore(m=base,color='blue',marker_kwds=dict(radius=5))

### Exercise 3

<div class="alert-success">
    
1. Create a HULL for some set of line map.

2. Compute the distance matrix between the HULLS and a map of points.

3. Plot the HULLS and the points. Show the closest and farthest points to the HULL.
    
</div>    

## Distances using _Buffers_

A very important case in distance analysis is the use of buffers:

In [ ]:
# remember:
distanceMatrixKM_riv_air

In [ ]:
# getting a value (it can be any value)
distanceMatrixKM_riv_air.loc['Amazon'].min() # I chose min

We can use any value to create a buffer:

In [ ]:
minMts=distanceMatrixKM_riv_air.loc['Amazon'].min()*1000 # km

#the buffer is a polygon:
rivers[rivers.NAME=='Amazon'].buffer(distance = minMts)

In [ ]:
# see buffer:
bufferAroundAmazon=rivers[rivers.NAME=='Amazon'].buffer(distance = minMts)
bufferAsBase=bufferAroundAmazon.explore(color='red')
rivers[rivers.NAME=='Amazon'].explore(m=bufferAsBase,color='blue',style_kwds={'weight':0.5})

Above we used the buffer (red polygon), and the river (blue). Let me add a layer of airports (small ones):

In [ ]:
small_airports=airports[airports['kind']=='small_airport']

# plotting
rivers[rivers.NAME=='Amazon'].explore(m=bufferAsBase,color='blue',style_kwds={'weight':0.5})
small_airports.explore(m=bufferAsBase,color='black')

Now, we can use the buffer (polygon) to keep the airports that are at that particular distance around the river:

In [ ]:

riversWithinBuffer=small_airports.clip(mask=bufferAroundAmazon)
#
riversWithinBuffer

In [ ]:
# plotting the airports within buffer
bufferAsBase=bufferAroundAmazon.explore(color='red')
rivers[rivers.NAME=='Amazon'].explore(m=bufferAsBase,color='blue',style_kwds={'weight':0.5})
riversWithinBuffer.explore(m=bufferAsBase,color='black')

In [ ]:
# minimum of all the minimum by row
distanceMatrixKM_riv_air.min(axis=1).min()

In [ ]:
# using the previous value
minMinMts_5=5*distanceMatrixKM_riv_air.min(axis=1).min()*1000


allMinBuffer=rivers.buffer(distance = minMinMts_5).explore(color='red')
rivers.explore(m=allMinBuffer,color='blue',style_kwds={'weight':0.5})

In [ ]:
# you see all the buffer polygons:
riversAll_buf=rivers.buffer(distance = minMinMts_5)
riversAll_buf

Now keep small airports in buffer:

In [ ]:
allRiversWithinBuffs=small_airports.clip(riversAll_buf)
allRiversWithinBuffs

In [ ]:
# simple
base=riversAll_buf.plot(color='yellow')
allRiversWithinBuffs.plot(ax=base, color='green', markersize=1)

In [ ]:
# folium

base=riversAll_buf.explore(color='yellow')
allRiversWithinBuffs.explore(m=base, color='green')

### Exercise 4

<div class="alert-success">
    
1. Select a line map and a point one.

2. Get the buffer for the lines, select a distance.

3. Keep the points that are within the buffer (you might need to play with differn distances until you show something interesting.  
    
</div>   

# II. MINING SPATIAL DATA

It is time to use  *dataPeru_indicadores.xlsx* file and the  map of Peruvian districts: *DistritosMap.zip* (zipped shape file).

Let's read the data in :

In [ ]:
# data table
import pandas as pd
import os

peruDataLink="https://github.com/CienciaDeDatosEspacial/GeoDataFrame_Analytics/raw/main/data/dataPeru_indicadores.xlsx"
datadis=pd.read_excel(peruDataLink,
                     dtype={'Ubigeo': object})
datadis.info()

In [ ]:
datadis.head()

In [ ]:
# map
import geopandas as gpd

peruMapaDistLink="https://github.com/CienciaDeDatosEspacial/GeoDataFrame_Analytics/raw/main/maps/DistritosMap.zip"

mapdis=gpd.read_file(peruMapaDistLink)

mapdis.info()

In [ ]:
mapdis.head()

Next we will merge the indicators table into the map.

## Merging

When merging, verify the amount of resulting rows first:

In [ ]:
mapdis.merge(datadis, left_on='DISTRITO', right_on='Distrito').shape

The amount of rows increases when merged. Let's do some preprocessing:

1. Capitalization:

In [ ]:
# all capitals, no empty spaces before or after.

capitalizeColumns=lambda x: x.str.upper().str.strip()
datadis[['Provincia','Distrito']]=datadis[['Provincia','Distrito']].apply(capitalizeColumns)
mapdis[['PROVINCIA','DISTRITO']]=mapdis[['PROVINCIA','DISTRITO']].apply(capitalizeColumns)

2. Spanish symbols: The names may come with some symbols that may cause trouble (', ~). Let's get rid of those:

In [ ]:
import unidecode


byePunctuation=lambda x: unidecode.unidecode(x)
datadis[['Provincia','Distrito']]=datadis[['Provincia','Distrito']].map(byePunctuation)  #applymap for olderpandas
mapdis[['PROVINCIA','DISTRITO']]=mapdis[['PROVINCIA','DISTRITO']].map(byePunctuation) #applymap for olderpandas

3. Check uniqueness

In [ ]:
datadis.Distrito.duplicated().sum(),mapdis.DISTRITO.duplicated().sum()

The presence of duplicates, forces we create  a column of unique values:

In [ ]:
# concatenating
datadis['provDist']=["+".join(pd) for pd in zip (datadis.Provincia,datadis.Distrito)]
mapdis['provDist']=["+".join(pd) for pd in zip (mapdis.PROVINCIA,mapdis.DISTRITO)]

In [ ]:
# the new column looks like this:
datadis['provDist'].head()

Let's find out what is NOT matched between the  tables:

In [ ]:
nomatch_df=set(datadis.provDist)- set(mapdis.provDist)
nomatch_gdf=set(mapdis.provDist)-set(datadis.provDist)

This is the amount of rows that could not be matched:

In [ ]:
len(nomatch_df), len(nomatch_gdf)

Let's try renaming the districts using **fuzzy merging**:

In [ ]:
# pick the closest match from nomatch_gdf for a value in nomatch_df
from thefuzz import process
[(dis,process.extractOne(dis,nomatch_gdf)) for dis in sorted(nomatch_df)]

If you are comfortable, you prepare a _dictionary_ of changes:

In [ ]:
# is this OK?

changesDis_df={dis:process.extractOne(dis,nomatch_gdf)[0] for dis in sorted(nomatch_df)}
changesDis_df

Now, make the replacements:

In [ ]:
datadis.replace({'provDist':changesDis_df},inplace=True)

Now the merge can happen:

In [ ]:
datadisMap=mapdis.merge(datadis, on='provDist')
# check
datadisMap.info()

In [ ]:
bye=['Departamento', 'Provincia', 'Distrito','INSTITUCIO','provDist']
datadisMap.drop(columns=bye,inplace=True)

# keeping
datadisMap.head()

### Exercises Part II

<div class="alert-warning">
    
Exercises 5,6,7,8, and 9 for this part must read you data from GitHub links, and the coding and results must be published using GitHub Pages. This is a different project, so those exercises will be published in a different webpage.

</div>

### Exercise 5

<div class="alert-success">
    
1. Get a polygons map of the lowest administrative unit possible.
    
2. Get a table of variables for those units. At least 3 numerical variables.

3. Preprocess both tables and get them ready for merging.

4. Do the merging, making the changes needed so that you keep the most columns.
    
    
</div>

## Mining one variable

In the session on [Intro to GeoDF](https://cienciadedatosespacial.github.io/intro_geodataframe/) we did a lot on this. The main idea was simply to know the behavior of one variable, and plot it as a choropleth map.

In this case, **spatial properties** of the data were _NOT_ used at all, for example:

1) Descriptive stats would be same in a simple data frame:

In [ ]:
# statistics
datadisMap.IDH2019.describe()

2) A histogram would be same in a simple data frame:

In [ ]:
import seaborn as sea

sea.histplot(datadisMap.IDH2019, color='yellow')

3. Transform and Discretize: We also learned that we could rescale and discretize. But, given this behavior, bell-shaped,  we just need to discretize; which I will simply do using the _fisherjenks_ scheme:

In [ ]:
datadisMap.explore(
    column="IDH2019",
    scheme="fisherjenks",
    legend=True,
    tooltip=False,
    popup=['DEPARTAMEN', 'PROVINCIA', 'DISTRITO'],  # show popup (on-click)
    legend_kwds=dict(colorbar=False)
)

## Spatial Properties: determining the _neighborhood_

We can compute the neighborhood for each object in a map using different options:

1. The polygons that share borders:

In [ ]:
from libpysal.weights import Queen, Rook, KNN

# rook
w_rook = Rook.from_dataframe(datadisMap,use_index=False)

In [ ]:
w_rook.islands

2. The polygons that share at least a point:

In [ ]:
# queen
w_queen = Queen.from_dataframe(datadisMap,use_index=False)

In [ ]:
w_queen.islands

Let me show the islands detected in the previous steps:

In [ ]:
datadisMap.iloc[w_queen.islands,:].explore()

The presence of _islands_ will be problematic in more complex applications. An alternative is:

3) Nearest neighbors:

In [ ]:
# k=8 nearest neighbors
w_knn8 = KNN.from_dataframe(datadisMap, k=8)

In [ ]:
w_knn8.islands

Let's understand the differences:

In [ ]:
# first district in the GDF:
datadisMap.head(1)

In [ ]:
# amount of neighbors of that district
len(w_rook.neighbors[0])

In [ ]:
# details
datadisMap.iloc[w_rook.neighbors[0],]

In [ ]:
# see the neighbor
datadisMap.iloc[w_rook.neighbors[0] ,].plot(facecolor="yellow")

In [ ]:
# see whole area
base=datadisMap[datadisMap.PROVINCIA=="TACNA"].plot()
datadisMap.iloc[w_rook.neighbors[0] ,].plot(ax=base,facecolor="yellow",edgecolor='k')
datadisMap.head(1).plot(ax=base,facecolor="red")

Let's do the same with queen neighbors:

In [ ]:
# how many
len(w_queen.neighbors[0])

In [ ]:
# details
datadisMap.iloc[w_queen.neighbors[0] ,]

In [ ]:
# see
datadisMap.iloc[w_queen.neighbors[0] ,].plot(facecolor="yellow")

In [ ]:
# whole area
base=datadisMap[datadisMap.PROVINCIA=="TACNA"].plot()
datadisMap.iloc[w_queen.neighbors[0] ,].plot(ax=base,facecolor="yellow",edgecolor='k')
datadisMap.head(1).plot(ax=base,facecolor="red")

What about the _eight_ closest ones?

In [ ]:
w_knn8.neighbors[0]

In [ ]:
base=datadisMap[datadisMap.PROVINCIA=="TACNA"].plot()
datadisMap.iloc[w_knn8.neighbors[0],].plot(ax=base,facecolor="yellow")
datadisMap.head(1).plot(ax=base,facecolor="red")

In [ ]:
# what about k=4

w_knn4 = KNN.from_dataframe(datadisMap, k=4)

base=datadisMap[datadisMap.PROVINCIA=="TACNA"].plot()
datadisMap.iloc[w_knn4.neighbors[0],].plot(ax=base,facecolor="yellow")
datadisMap.head(1).plot(ax=base,facecolor="red")

### Exercise 6

<div class="alert-success">
        
Compute the neighbors of the capital of your country. Plot the results for each of the options.
    
</div>

## Global spatial correlation

If a spatial unit (a row) value in a variable is correlated with values of the neighbors, you know that proximity is interfering with the interpretation.

We need the neighboorhood matrix (the weight matrix) to compute spatial correlation.

In [ ]:
pd.DataFrame(*w_knn8.full()) # 1 means both are neighbors

If we standardize by row, the neighboor adds to 1:

In [ ]:
# needed for spatial correlation
w_knn8.transform = 'R'

In [ ]:
# after transformation
pd.DataFrame(*w_knn8.full()).head(12).sum(axis=1)

Spatial correlation is measured by the Moran's I statistic:

In [ ]:
from esda.moran import Moran

moranIDH = Moran(datadisMap['IDH2019'], w_knn8)
moranIDH.I,moranIDH.p_sim

A significant Moran's I suggest spatial correlation. Let's see the spatial scatter plot

In [ ]:
from splot.esda import moran_scatterplot

fig, ax = moran_scatterplot(moranIDH, aspect_equal=True)
ax.set_xlabel('IDH_std')
ax.set_ylabel('SpatialLag_IDH_std');

### Exercise 7

<div class="alert-success">
    
1. Compute the Moran's coefficient for **one** of your three  numeric variables.
    
2. Make a scatter plot for each variable.
    
</div>

## Local Spatial Correlation

We can compute a Local Index of Spatial Association (LISA -local Moran) for each map object. That will help us find spatial clusters (spots) and spatial outliers:

* A **hotSpot** is a polygon whose value in the variable is high AND is surrounded with polygons with also high values.

* A **coldSpot** is a polygon whose value in the variable is low AND is surrounded with polygons with also low values.

* A **coldOutlier** is a polygon whose value in the variable is low BUT is surrounded with polygons with  high values.

* A **hotOutlier** is a polygon whose value in the variable is high BUT is surrounded with polygons with  low values.

It is also possible that no significant correlation is detected. Let's see those values:

In [ ]:
# A LISA for each district using IDH2019
from esda.moran import Moran_Local
lisaIDH = Moran_Local(y=datadisMap['IDH2019'], w=w_knn8,seed=2022)

In [ ]:
fig, ax = moran_scatterplot(lisaIDH,p=0.05)
ax.set_xlabel('IDH_std')
ax.set_ylabel('SpatialLag_IDH_std');

You find that a district is in a **quadrant**. If the district is NOT grey, then the LISA is significant. Let's represent that information in a map:

In [ ]:
import matplotlib.pyplot as plt

from splot.esda import lisa_cluster
f, ax = plt.subplots(1, figsize=(12, 12))
plt.title('Spots and Outliers')
fig = lisa_cluster(lisaIDH,
                   datadisMap,ax=ax,
                   legend_kwds={'loc': 'center left',
                                'bbox_to_anchor': (0.7, 0.6)});
plt.show()

Let me add the informtion in lisaIDH to the GeoDF:

In [ ]:
# quadrant, # significance
lisaIDH.q, lisaIDH.p_sim

In [ ]:
# quadrant: 1 HH,  2 LH,  3 LL,  4 HL
pd.Series(lisaIDH.q).value_counts()

The info in **lisaIDH.q** can not be used right away, we need to add if the local spatial correlation is significant:

In [ ]:
datadisMap['IDH_quadrant']=[l if p <0.05 else 0 for l,p in zip(lisaIDH.q,lisaIDH.p_sim)  ]
datadisMap['IDH_quadrant'].value_counts()

Now, we recode:

In [ ]:
labels = [ '0 no_sig', '1 hotSpot', '2 coldOutlier', '3 coldSpot', '4 hotOutlier']

datadisMap['IDH_quadrant_names']=[labels[i] for i in datadisMap['IDH_quadrant']]

datadisMap['IDH_quadrant_names'].value_counts()

In [ ]:

# custom colors
from matplotlib import colors
myColMap = colors.ListedColormap([ 'white', 'pink', 'cyan', 'azure','red'])

# Set up figure and ax
f, ax = plt.subplots(1, figsize=(12,12))
# Plot unique values choropleth including
# a legend and with no boundary lines

plt.title('Spots and Outliers')

datadisMap.plot(column='IDH_quadrant_names',
                categorical=True,
                cmap=myColMap,
                linewidth=0.1,
                edgecolor='k',
                legend=True,
                legend_kwds={'loc': 'center left',
                             'bbox_to_anchor': (0.7, 0.6)},
                ax=ax)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

### Exercise 8

<div class="alert-success">
    
1. Compute the Local Moran for the variables in your data that have significant spatial correlation.
    
2. Create a new column for each of those variables, with a label ('0 no_sig', '1 hotSpot', '2 coldOutlier', '3 coldSpot', '4 hotOutlier').

3. Prepare a map for each of the variables analyzed, showing the spots and outliers.
    
</div>

## Mining several variables

Let me select some columns:

In [ ]:
selected_variables = ['Educ_sec_comp2019_pct',
                     'NBI2017_pct',
                     'Viv_sin_serv_hig2017_pct']

In [ ]:
# see distribution
sea.boxplot(datadisMap[selected_variables])

Let me check their monotony:

In [ ]:
datadisMap[selected_variables].corr()

In [ ]:
sea.pairplot(
    datadisMap[selected_variables], kind="reg", diag_kind="kde"
)

Here, we can reverse the values of *Educ_sec_comp2019_pct*. First let me standardize:

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
normalized_data = scaler.fit_transform(datadisMap[selected_variables])
sea.displot(pd.melt(pd.DataFrame(normalized_data,columns=selected_variables)),
            x="value", hue="variable",kind="kde",
            log_scale=(False,False))

Let me create new variables with the standardized values:

In [ ]:
# new names
selected_variables_new_std=[s+'_std' for s in selected_variables]

# add colunms
datadisMap[selected_variables_new_std]=normalized_data

Now, it is easy to reverse:

In [ ]:
datadisMap['Educ_sec_NO_comp2019_pct_std']=-1*datadisMap.Educ_sec_comp2019_pct_std

In [ ]:
# as a result:
selected_variables_new_std = ['Educ_sec_NO_comp2019_pct_std',
                     'NBI2017_pct_std',
                     'Viv_sin_serv_hig2017_pct_std']
sea.pairplot(
    datadisMap[selected_variables_new_std], kind="reg", diag_kind="kde"
)

### Conventional Clustering

Here, I will use the three variables to create clusters of districts. Let me explore how many clusters could be created:

In [ ]:
from scipy.cluster import hierarchy as hc


Z = hc.linkage(datadisMap[selected_variables_new_std], 'ward')
# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('cases')
plt.ylabel('distance')
hc.dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=1,  # font size for the x axis labels
)
plt.show()

The dendogram recommends three groups. Let me request six.

Let me use a common hierarchical technique following a agglomerative approach:

In [ ]:
from sklearn.cluster import AgglomerativeClustering as agnes

import numpy as np
np.random.seed(12345)# Set seed for reproducibility

# Initialize the algorithm, requesting 6 clusters
model = agnes(linkage="ward", n_clusters=6).fit(datadisMap[selected_variables_new_std])

# Assign labels to main data table
datadisMap["hc_ag6"] = model.labels_

In [ ]:
# see distribution of districts
datadisMap["hc_ag6"].value_counts()

We could try to find the pattern that created the clusters:

In [ ]:
datadisMap.groupby("hc_ag6")[selected_variables_new_std].mean()

Let me show you the six groups of districts which have similar behavior in three variables:

In [ ]:
# Set up figure and ax
f, ax = plt.subplots(1, figsize=(9, 9))
# Plot unique values choropleth including
# a legend and with no boundary lines
datadisMap.plot(
    column="hc_ag6", categorical=True, legend=True, linewidth=0, ax=ax
)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

### Regionalization: Spatial Clustering

Spatial clustering or Regionalization will force the contiguity of the polygons to make a cluster.

In [ ]:
# modify previous funtion call to specify cluster model with spatial constraint

model_queen = agnes(linkage="ward",
                    n_clusters=6,
                    connectivity=w_queen.sparse).fit(datadisMap[selected_variables_new_std])
# Fit algorithm to the data
datadisMap["hc_ag6_wQueen"] = model_queen.labels_

We knew this would happen because we have islands. Then this results may not be satisfactory:

In [ ]:
# Set up figure and ax
f, ax = plt.subplots(1, figsize=(9, 9))
# Plot unique values choropleth including a legend and with no boundary lines
datadisMap.plot(
    column="hc_ag6_wQueen",
    categorical=True,
    legend=True,
    linewidth=0,
    ax=ax,
)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

We have a couple of KNN weight matrices. Let's use those instead:

In [ ]:
model_wknn8 = agnes(linkage="ward",
                    n_clusters=6,
                    connectivity=w_knn8.sparse).fit(datadisMap[selected_variables_new_std])
datadisMap["hc_ag6_wknn8"] = model_wknn8.labels_


model_wknn4 = agnes(linkage="ward",
                    n_clusters=6,
                    connectivity=w_knn4.sparse).fit(datadisMap[selected_variables_new_std])
datadisMap["hc_ag6_wknn4"] = model_wknn4.labels_

In [ ]:
# Set up figure and ax
f, ax = plt.subplots(1, figsize=(10, 12))
# Plot unique values choropleth including a legend and with no boundary lines
datadisMap.plot(
    column="hc_ag6_wknn8",
    categorical=True,
    legend=True,
    linewidth=0,
    ax=ax,
)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

In [ ]:
# Set up figure and ax
f, ax = plt.subplots(1, figsize=(10, 12))
# Plot unique values choropleth including a legend and with no boundary lines
datadisMap.plot(
    column="hc_ag6_wknn4",
    categorical=True,
    legend=True,
    linewidth=0,
    ax=ax,
)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

We could evaluate two aspects of these clustering results:

* “Compactness” of cluster shape, using the isoperimetric quotient (IPQ). This compares the area of the region to the area of a circle with the same perimeter as the region. For this measure, more compact shapes have an IPQ closer to 1, whereas very elongated or spindly shapes will have IPQs closer to zero. For the clustering solutions, we would expect the IPQ to be very small indeed, since the perimeter of a cluster/region gets smaller the more boundaries that members share.

In [ ]:
from esda import shape as shapestats
results={}
for cluster_type in ("hc_ag6_wknn4", "hc_ag6_wknn8", "hc_ag6"):
    # compute the region polygons using a dissolve
    regions = datadisMap[[cluster_type, "geometry"]].to_crs(24892).dissolve(by=cluster_type)
    # compute the actual isoperimetric quotient for these regions
    ipqs = shapestats.isoperimetric_quotient(regions)
    # cast to a dataframe
    result = {cluster_type:ipqs}
    results.update(result)
# stack the series together along columns
pd.DataFrame(results)

An alternative could be _convex_hull_ratio_, simply the division of the area of the cluster by the area of its convex hull.

In [ ]:
from esda import shape as shapestats
results={}
for cluster_type in ("hc_ag6_wknn4", "hc_ag6_wknn8", "hc_ag6"):
    # compute the region polygons using a dissolve
    regions = datadisMap[[cluster_type, "geometry"]].to_crs(24892).dissolve(by=cluster_type)
    # compute the actual convex hull quotient for these regions
    chullr = shapestats.convex_hull_ratio(regions)
    # cast to a dataframe
    result = {cluster_type:chullr}
    results.update(result)
# stack the series together along columns
pd.DataFrame(results)

In both cases, the non spatial clusters do better.

* Goodness of fit. Here we have two metrics:
    - metrics.calinski_harabasz_score
    - silhouette_score

In [ ]:
from sklearn import metrics

fit_scores = []
for cluster_type in ("hc_ag6_wknn4", "hc_ag6_wknn8", "hc_ag6"):
    # compute the CH score
    ch_score = metrics.calinski_harabasz_score(
        # using scaled variables
        datadisMap[selected_variables_new_std],
        # using these labels
        datadisMap[cluster_type],
    )
    sil_score = metrics.silhouette_score(
        # using scaled variables
        datadisMap[selected_variables_new_std],
        # using these labels
        datadisMap[cluster_type],
    )
    # and append the cluster type with the CH score
    fit_scores.append((cluster_type, ch_score,sil_score))


# re-arrange the scores into a dataframe for display
pd.DataFrame(
    fit_scores, columns=["cluster type", "CH score", "SIL score"]
).set_index("cluster type")

Again, the conventional clustering beats the others, as you want bigger values in both.

### Exercise 9

<div class="alert-success">
    
Use your three variables to carry out the cluster/regional analysis.
    
</div>


### Conventional Regression



In [ ]:
from pysal.model import spreg

dep_var_name=['NBI2017_pct']
ind_vars_names=['Educ_sec_comp2019_pct','Viv_sin_serv_hig2017_pct']


ols_model = spreg.OLS(
    # Dependent variable
    datadisMap[dep_var_name].values,
    # Independent variables
    datadisMap[ind_vars_names].values,
    w=w_knn8,
    spat_diag = True,
    moran=True,
    # Dependent variable name
    name_y=dep_var_name[0],
    # Independent variable name
    name_x=ind_vars_names)

print(ols_model.summary)

### Spatial Regression

* Spatial Lag Regression

In [ ]:
moranNBI = Moran(datadisMap[dep_var_name], w_knn8)
moranNBI.I,moranNBI.p_sim

In [ ]:
fig, ax = moran_scatterplot(moranNBI, aspect_equal=True)
ax.set_xlabel('NBI')
ax.set_ylabel('SpatialLag_NBI');

In [ ]:
lag_model = spreg.ML_Lag(
    # Dependent variable
    datadisMap[dep_var_name].values,
    # Independent variables
    datadisMap[ind_vars_names].values,
    w=w_knn8,
    # Dependent variable name
    name_y=dep_var_name[0],
    # Independent variable name
    name_x=ind_vars_names
    )

print(lag_model.summary)

* Spatial Error Regression

In [ ]:

moranError = Moran(ols_model.u, w_knn8)
moranError.I,moranError.p_sim

In [ ]:
fig, ax = moran_scatterplot(moranError, aspect_equal=True)
ax.set_xlabel('OlsError')
ax.set_ylabel('SpatialOlsError');

In [ ]:
err_model = spreg.ML_Error(
    # Dependent variable
    datadisMap[dep_var_name].values,
    # Independent variables
    datadisMap[ind_vars_names].values,
    w=w_knn8,
    # Dependent variable name
    name_y=dep_var_name[0],
    # Independent variable name
    name_x=ind_vars_names
    )

print(err_model.summary)

* Spatial Error Regression, correcting heteroscedasticy.

In [ ]:
error_Het_model = spreg.GM_Error_Het(
    # Dependent variable
    datadisMap[dep_var_name].values,
    # Independent variables
    datadisMap[ind_vars_names].values,
    # Spatial weights matrix
    w=w_knn8,
    # Dependent variable name
    name_y=dep_var_name[0],
    # Independent variable name
    name_x=ind_vars_names,
)
print(error_Het_model.summary)

### Exercise 10

<div class="alert-success">
    
Use your three variables to carry out regression analysis (conventional and spatial).
    
</div>